In [1]:
# The ultimate target feature: time from one stop to another

# Imports
import pandas as pd
import numpy as np
from datetime import date, datetime
from patsy import dmatrices
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.tree import export_graphviz, DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib 
from statsmodels.formula.api import ols

# Read csv file into a dataframe.
df = pd.read_csv('csv_data/route4.csv')

/Users/reggiemurphy/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 1.0 Data Cleaning 

## 1.1 Rename column names

In [2]:
df = df.rename(columns={'Timeframe': 'Start_date'})

## 1.2 Dropping duplicates

In [3]:
df=df.drop_duplicates(keep='first')

## 1.3 Dropping constant columns or columns with missing data

In [4]:
df = df.drop('Direction', axis=1)
df = df.drop('Unnamed: 0', axis=1)
df = df.drop('Congestion', axis=1)
df[df.Journey_Pattern_ID == 'null']
df = df[df['Journey_Pattern_ID'] != '00040002']
df = df[df['Journey_Pattern_ID'] != '00041002']

## 1.4 Remove rows where bus is not at stop

In [5]:
df = df.loc[(df != 0).all(axis=1), :]

## 1.5 Group to normalise time & remove rows where bus idle at stop

In [6]:
# Create empty column which will hold normalised time
df['normal_time'] = 0

In [7]:
# Create empty column which will hold the stop order
df['stop_order'] = 0

In [8]:
grouped_df = df.groupby(['Vehicle_Journey_ID', 'Start_date'])

In [9]:
def normalize_time(df):
    """Normalise the time for each journey"""
    for i in range(df['Timestamp'].size):
        df['normal_time'].values[i] = (df['Timestamp'].values[i] - df['Timestamp'].values[0]) / 1000000
        
    return df

In [10]:
norm_gb = grouped_df.apply(normalize_time)

In [11]:
grouped_df = norm_gb.groupby(['Vehicle_Journey_ID', 'Start_date'])

In [12]:
def remove_idle_at_stop(df):
    df = df.drop_duplicates(subset='Stop_ID', keep='first')
    
    return df

In [13]:
norm_gb = grouped_df.apply(remove_idle_at_stop)

In [14]:
grouped_df = norm_gb.groupby(['Vehicle_Journey_ID', 'Start_date'])

/Users/reggiemurphy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: 'Vehicle_Journey_ID' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  if __name__ == '__main__':
/Users/reggiemurphy/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: 'Start_date' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  if __name__ == '__main__':


In [15]:
def route_order(df):
    for i in range(df['Timestamp'].size):
        df['stop_order'].values[i] = i
    
    return df

In [16]:
df = grouped_df.apply(route_order)

In [17]:
# grouped_df = norm_gb.groupby(['Vehicle_Journey_ID', 'Start_date'])

In [18]:
# grouped_df.get_group((4601, '2013-01-29'))

## 1.6 Add new features

In [19]:
df["Time"] = pd.to_datetime(df['Timestamp']*1000, unit="ns")

In [20]:
df['IsWeekend'] = np.where((df['Time'].dt.dayofweek > 5), 1, 0)

In [21]:
df['HourOfDay'] = df['Time'].dt.hour

In [22]:
df['IsEveningTime'] = np.where((df['Time'].dt.hour > 17), 1, 0)

In [23]:
df['IsPeakTime'] = np.where(((df['HourOfDay'] >= 7) & (df['HourOfDay'] <= 10)) | ((df['HourOfDay'] >= 16) & (df['HourOfDay'] <= 19)), 1, 0)

In [24]:
df['SchoolHoliday'] = np.where((df['Time'].dt.date == datetime(2012, 11, 1).date()) | (

df['Time'].dt.date == datetime(2012, 11, 2).date()) | (

                                       df['Time'].dt.date == datetime(2013, 1, 1).date()) | (

                                       df['Time'].dt.date == datetime(2013, 1, 2).date()) | (

                                       df['Time'].dt.date == datetime(2013, 1, 3).date()) | (

                                       df['Time'].dt.date == datetime(2013, 1, 4).date()), 1, 0)

# 2.0 Merge Datasets

## 2.1 Merge bus stop info

In [25]:
# df=df.drop_duplicates(subset='Time', keep='first')

In [26]:
df_bus_stop = pd.read_csv('csv_data/busstopinfo.csv', encoding='latin-1')

In [27]:
df_bus_stop = df_bus_stop.rename(columns={'stopid': 'Stop_ID'})
df_bus_stop = df_bus_stop.rename(columns={'fullname': 'Stop_name'})

In [28]:
df_bus_stop = df_bus_stop[['Stop_ID', 'Stop_name']]

In [29]:
df = pd.merge(df, df_bus_stop, on=['Stop_ID'])

In [30]:
# df_bus_stop_route4 = df_bus_stop.loc[(df_bus_stop.route1 == '4')]
# df_bus_stop_route4.shape

In [31]:
# df_southbound = df.loc[(df.Vehicle_ID == 43048) & (df.Vehicle_Journey_ID == 4909)]

In [32]:
# df[(df.Vehicle_ID == 43048)]["Vehicle_Journey_ID"].unique()

Get northbound order of stops

In [33]:
# df_northbound = df.loc[(df.Vehicle_ID == 43048) & (df.Vehicle_Journey_ID == 4572)]

In [34]:
# df_bus_stop_route4 = df_bus_stop_route4[['Stop_ID', 'fullname']]
# df_bus_stop_route4.columns

In [35]:
# df_south = df_southbound.drop_duplicates(subset='Stop_ID', keep='first')

In [36]:
# df_south['stop_order'] = df_south.groupby(['Vehicle_Journey_ID', 'At_Stop', 'Start_date', 'Vehicle_ID'])['Timestamp','Stop_ID'].cumcount().reindex()

In [37]:
# df_north = df_northbound.drop_duplicates(subset='Stop_ID', keep='first')

In [38]:
# df_north['stop_order'] = df_north.groupby(['Vehicle_Journey_ID', 'At_Stop', 'Start_date', 'Vehicle_ID'])['Timestamp','Stop_ID'].cumcount().reindex()

In [39]:
# df_order = df_south[['Stop_ID', 'stop_order']]
# df_order = df_order.drop_duplicates(subset='Stop_ID', keep='first')

In [40]:
# df_order2 = df_north[['Stop_ID', 'stop_order']]
# df_order2=df_order2.drop_duplicates(subset='Stop_ID', keep='first')

In [41]:
# frames = [df_order, df_order2]
# df_order3 = pd.concat(frames)

In [42]:
# df_bus_stop_route4= pd.merge(df_order3, df_bus_stop_route4, on=['Stop_ID'])

## 2.2 Merge weather data

In [43]:
df_weather = pd.read_csv('csv_data/weather_data.csv', encoding='latin-1')

In [44]:
df_weather = df_weather.rename(columns={'Rainfall (Yes or No)': 'Rain'})
df_weather = df_weather.rename(columns={'Temperature C': 'Temperature'})
df_weather = df_weather.rename(columns={'Relative Humidity (%)': 'Humidity'})
df_weather = df_weather.rename(columns={'Over 1mm Rain?': 'Heavy_rain'})
df_weather = df_weather.rename(columns={'Precipitation (mm)': 'Precipitation'})
df_weather = df_weather.rename(columns={'Date': 'Time'})
df_weather['Time'] = pd.to_datetime(df_weather['Time'])
df_weather.sort_values(['Time'], ascending=[True], inplace=True)
df_weather['HourOfDay'] = df_weather['Time'].dt.hour

In [45]:
df.sort_values(['Time'], ascending=[True], inplace=True)
df_weather.sort_values(['Time'], ascending=[True], inplace=True)
df =  pd.merge_asof(df, df_weather, on='Time')

In [46]:
#code from here: https://stackoverflow.com/questions/27313647/merging-two-pandas-dataframes-results-in-duplicate-columns

def drop_y(df):
    # list comprehension of the cols that end with '_y'
    to_drop = [x for x in df if x.endswith('_y')]
    df.drop(to_drop, axis=1, inplace=True)

drop_y(df)
df.columns

Index(['Timestamp', 'LineID', 'Journey_Pattern_ID', 'Start_date',
       'Vehicle_Journey_ID', 'Operator', 'Lon', 'Lat', 'Delay', 'Block_ID',
       'Vehicle_ID', 'Stop_ID', 'At_Stop', 'normal_time', 'stop_order', 'Time',
       'IsWeekend', 'HourOfDay_x', 'IsEveningTime', 'IsPeakTime',
       'SchoolHoliday', 'Stop_name', 'Precipitation', 'Temperature',
       'Vapour Pressure (hPa)                ', 'Humidity',
       'Mean Sea Level Pressure (hPa)', 'Rain', 'Heavy_rain'],
      dtype='object')

In [47]:
def rename_x(df):
    for col in df:
        if col.endswith('_x'):
            df.rename(columns={col:col.rstrip('_x')}, inplace=True)
rename_x(df)

# 3.0 Remove and categories columns

## 3.1 Drop missing values

In [48]:
df = df[df['Journey_Pattern_ID'] != 'null']

## 3.2 Categorise to continuous and categorical

In [49]:
df['IsWeekend'] = df['IsWeekend'].astype('category')
df['HourOfDay'] = df['HourOfDay'].astype('category')
df['IsEveningTime'] = df['IsEveningTime'].astype('category')
df['SchoolHoliday'] = df['SchoolHoliday'].astype('category')
df['IsPeakTime'] = df['IsPeakTime'].astype('category')
df['Operator'] = df['Operator'].astype('category')
df['At_Stop'] = df['At_Stop'].astype('category')
categorical_columns = df.select_dtypes(['category']).columns

In [50]:
continuous_columns = df.select_dtypes(['int64', 'float64']).columns

## 3.3 Drop columns no longer needed

In [51]:
df = df.drop('Lat', axis=1)
df = df.drop('Lon', axis=1)
df = df.drop('Block_ID', axis=1)
df = df.drop('Operator', axis=1)

## 3.4 Create time to destination feature

In [52]:
df['end_time'] = df.groupby(['stop_order','Journey_Pattern_ID', 'Start_date', 'HourOfDay'])['Timestamp'].transform(max)

In [53]:
df['time_diff_to_destination'] = df['end_time'] - df['Timestamp']

In [54]:
df['distance_to_end'] = df.stop_order.max() - df.stop_order

In [55]:
df.columns

Index(['Timestamp', 'LineID', 'Journey_Pattern_ID', 'Start_date',
       'Vehicle_Journey_ID', 'Delay', 'Vehicle_ID', 'Stop_ID', 'At_Stop',
       'normal_time', 'stop_order', 'Time', 'IsWeekend', 'HourOfDay',
       'IsEveningTime', 'IsPeakTime', 'SchoolHoliday', 'Stop_name',
       'Precipitation', 'Temperature', 'Vapour Pressure (hPa)                ',
       'Humidity', 'Mean Sea Level Pressure (hPa)', 'Rain', 'Heavy_rain',
       'end_time', 'time_diff_to_destination', 'distance_to_end'],
      dtype='object')

# 4.0 Save DF to be used in Model Analysis

In [56]:
# Save cleaned dataframe to new CSV file
df.to_csv('csv_data/bus_route4_clean.csv', index=False)

---

In [57]:
# df_train = df[:130000]

In [58]:
# df_test = df[130000:]

In [59]:
# df_group = df.groupby(['Vehicle_Journey_ID', 'Start_date'])

In [60]:
# for k, g in df_group:
#     model = ols('time_diff_to_destination ~ distance_to_end + Rain + Temperature +Humidity + IsWeekend +HourOfDay + SchoolHoliday', g)
#     results = model.fit()
#     print(results.summary())